In [1]:
import cv2
from tkinter import Tk
from tkinter.filedialog import askopenfilename
from detect import run_detection
from pathlib import Path
from yolov5.models.common import DetectMultiBackend
import sys
import os
from yolov5.utils.torch_utils import select_device
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [9]:
root = Tk()
root.withdraw()
filepath = askopenfilename()
print(filepath)

/home/bms-del083/Documents/PythonScript/Deteksi_Plat/data/MVI_8993.MOV


In [3]:
def imshow(img,window_name='image',key=0):
    cv2.imshow(window_name,img)
    cv2.waitKey(key)
    cv2.destroyAllWindows()

In [4]:
def prepare(jenis):
    ROOT = 'yolov5'
    if str(ROOT) not in sys.path:
        sys.path.append(str(ROOT))
    ROOT = Path(os.path.relpath(ROOT, Path.cwd()))
    device = select_device('')
    path_weight = 'config/motor.pt'
    path_title = 'config/data_motor.yaml'
    if jenis == 'plat':
        path_weight = 'config/plat.pt'
        path_title = 'config/data_plat.yaml'
    return DetectMultiBackend(path_weight, device=device, dnn=False, data=path_title, fp16=False)

In [5]:
model_motor = prepare('motor')
model_plat = prepare('plat')

YOLOv5 🚀 2023-1-25 Python-3.9.12 torch-1.13.1+cu117 CPU

Fusing layers... 
Model summary: 214 layers, 7022326 parameters, 0 gradients, 15.9 GFLOPs
YOLOv5 🚀 2023-1-25 Python-3.9.12 torch-1.13.1+cu117 CPU

Fusing layers... 
Model summary: 214 layers, 7022326 parameters, 0 gradients, 15.9 GFLOPs


In [6]:
def boundedFrame(img,x,y,width,height):
    cv2.rectangle(img, (x, y), (x+width, y+height), (0, 0, 255), 2)
    submat = img[y:y+height, x:x+width]
    return submat, img

In [8]:
def detectHarris(m):
    gray = cv2.cvtColor(m, cv2.COLOR_BGR2GRAY)
    dst = cv2.cornerHarris(gray, blockSize=2, ksize=3, k=0.04)
    dst_norm = np.zeros(dst.shape, dtype=np.float32)
    return_mat = np.zeros(dst.shape, dtype=np.float32)
    cv2.normalize(dst, dst_norm, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    dst_norm_scaled = cv2.convertScaleAbs(dst_norm)
    for i in range(dst_norm.shape[0]):
        for j in range(dst_norm.shape[1]):
            if int(dst_norm[i,j]) > 150:
                cv2.circle(return_mat, (j,i), 5, (255,255,255), 2)
    return return_mat

In [9]:
def detectMSER(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    mser = cv2.MSER_create()
    regions, _ = mser.detectRegions(gray)
#     for contour in regions:
#         x, y, w, h = cv2.boundingRect(contour)
#         cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    text_regions = []
    for contour in regions:
        x, y, w, h = cv2.boundingRect(contour)
        text_regions.append((x,y,w,h))
#         text_regions.append(img[y:y+h, x:x+w])
    return text_regions

In [10]:
def find_max_area_contour(contours):
    max_area = 0
    max_area_index = 0
    for i, contour in enumerate(contours):
        area = cv2.contourArea(contour)
        if area > max_area:
            max_area = area
            max_area_index = i
    return max_area_index

def find_max_area_rect(rects):
    max_area = 0
    max_area_index = 0
    for i, rect in enumerate(rects):
        area = rect[2]*rect[3]
        if area > max_area:
            max_area = area
            max_area_index = i
    return max_area_index

In [24]:
def compareHarrisWithMSER(m):
    harris = detectHarris(m)
    mser = detectMSER(m)
    harris = harris.astype(np.uint8)
    contours, hierarchy = cv2.findContours(harris, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) > 0:
        r = cv2.boundingRect(contours[find_max_area_contour(contours)])
        kandidat_plat = []
        for region in mser :
            if (checkIsOverlaps(r, region)) :
                kandidat_plat.append(region)
        rect_plat = pilihPlat(kandidat_plat)
        if rect_plat != None:
            plat,m = boundedFrame(m,rect_plat[0],rect_plat[1],rect_plat[2],rect_plat[3])
            imshow(m)
            return m,plat
    return m,None

In [12]:
def pilihPlat(rects, toleransi = 0.5) :
    rasio_standart = 250/105 # rasio plat indonesia 250mm x 105mm
    rect_sesuai_rasio = []
    selisih_rasio = []
    for rect in rects:
        rasio = rect[2]/rect[3]
        selisih = abs(rasio-rasio_standart)
        if (selisih < toleransi):
            rect_sesuai_rasio.append(rect)
            selisih_rasio.append(selisih)
    plat_terpilih = None
    if len(rect_sesuai_rasio) > 0 :
        plat_terpilih = rect_sesuai_rasio[np.argmin(selisih_rasio)]
    return plat_terpilih

In [13]:
def checkIsOverlaps(rect1,rect2):
    x_overlap = max(0, min(int(rect1[0]) + int(rect1[2]), int(rect2[0]) + int(rect2[2])) - max(int(rect1[0]), int(rect2[0])))
    y_overlap = max(0, min(int(rect1[1]) + int(rect1[3]), int(rect2[1]) + int(rect2[3])) - max(int(rect1[1]), int(rect2[1])))
    overlap_area = x_overlap * y_overlap
    if overlap_area > 0:
        return True
    else:
        return False

In [37]:
def detection(frame,x,y,width,height):
    frame_asli = frame
    frame_roi,frame = boundedFrame(frame,x,y,width,height)
    path_roi = 'data/motor_detect.jpg'
    cv2.imwrite(path_roi, frame_roi)
    arr_loc_motor, frame_roi = run_detection(model_motor,path_roi,3)
    for loc in arr_loc_motor:
        object_asli = frame_asli[y+int(loc[1]):y+int(loc[3]), x+int(loc[0]):x+int(loc[2])]
        object_motor = frame_roi[int(loc[1]):int(loc[3]), int(loc[0]):int(loc[2])]
        path_object = 'data/plat_detect.jpg'
        cv2.imwrite(path_object,object_motor)
        loc_plat, object_plat = run_detection(model_plat,path_object,2)
#         for loc2 in loc_plat:
#             plat = object_asli[int(loc2[1]):int(loc2[3]), int(loc2[0]):int(loc2[2])]
            
#             y_top = y+int(loc[1])
#             x_top = x+int(loc[0])
#             resized_plat = cv2.resize(plat, (0, 0), fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
#             frame[y_top:y_top+resized_plat.shape[0], x_top:x_top+resized_plat.shape[1]] = resized_plat
        frame_roi[int(loc[1]):int(loc[3]), int(loc[0]):int(loc[2])] = object_plat
    frame[y:y+height, x:x+width] = frame_roi
    return frame

In [38]:
cap = cv2.VideoCapture(filepath)
if not cap.isOpened():
    print("Error opening video file")
idx = 0
while True:
    ret, frame = cap.read()
    if not ret:
        print("Video ended or error occurred")
        break
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    if (idx%(int(fps/4)) == 0) :
        cv2.namedWindow("Video", cv2.WINDOW_NORMAL)
        cv2.resizeWindow("Video", int((frame_width/frame_height)*700), 700)
        frame = detection(frame,100,550,1728,512)
        cv2.imshow("Video", frame)
    idx += 1
    key = cv2.waitKey(int(fps/8))
    if key == 27:
        break
cap.release()
cv2.destroyAllWindows()

In [40]:
arr = [[[2,3,4],[3,4,6]],[[2,3,4],[3,4,6]]]
print(np.array(arr).shape)
first_value = arr[:][:][0][:]
print(first_value)

(2, 2, 3)
[[2, 3, 4], [3, 4, 6]]


In [12]:
img = cv2.imread('data/plat_detect.jpg')
a = img[115:197, 117:293]
imshow(a)

In [11]:
arr, object_mat = run_detection(model_plat,'/home/bms-del083/Pictures/train_data_plat/images/train/tes14791.jpg')
print(arr)
imshow(object_mat)

[[tensor(0.), tensor(194.), tensor(165.), tensor(283.)]]


In [16]:
# def oldMSER(m):
#     gray = cv2.cvtColor(m, cv2.COLOR_BGR2GRAY)
#     mser = cv2.MSER_create(delta=3, min_area=10, max_area=100)
#     regions, _ = mser.detectRegions(gray)
#     return_mat = np.zeros(gray.shape, dtype=np.float32)
#     for region in regions:
#         hull = cv2.convexHull(region.reshape(-1, 1, 2))
#         cv2.polylines(return_mat, [hull], True, (255, 255, 255), 2)
#     return return_mat